<table class="ee-notebook-buttons" align="left">
    <td><a target="_blank"  href="https://github.com/ac-willeke/mapper-soilCondition"><img width=32px src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" style="filter: invert(100%)"/> View source on GitHub</a></td>
    <td><a target="_blank"  href="https://drive.google.com/drive/folders/1mEQBfa-tVViVWFt27XzUP4Wr19u1iuZm"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run in Google Colab</a></td>
</table>

# Mapping soil condition | Extract predictor variables using Google Earth Engine  

**Author:** Jenny Hanssen, Willeke A'Campo, Zander Venter

**Description:** Script to extract biological predictor variables for the wetland polygons using a Canopy Heigt Model from Norways national height model, Sentinel-1 and Sentinel-2 imagery. 



## Connect to Earth Engine 

In [1]:
# The earth engine api (ee) is standard in Google Colab
import ee
import subprocess
import pandas as pd

try:
    import geemap
    print("The packages are installed and imported.")
except ImportError:
    print('The geemap package is not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

The packages are installed and imported.


In [10]:
ee.Authenticate()


Successfully saved authorization token.


In [11]:
try:
    ee.Initialize() # Try to initialize earth engine
    print("Earth Engine is authenticated.")
except ee.EEException:
    ee.Authenticate() # Authenticate earth engine if initialization fails
    ##ee.Initialize() # init again 

Earth Engine is authenticated.


In [12]:
# import local module 
#from gee_functions import *

def resample(image, method, projection, maxPixels):
    resampled_image = image.reduceResolution(
            # Force the next reprojection to aggregate instead of resampling.
            reducer=method,
            maxPixels=maxPixels
        ).reproject(crs=projection) #Request the data at the scale and projection of the defined proejction image.
    
    return resampled_image

## Import Wetland Polygons

In [13]:
# Import mires fc
mires = ee.FeatureCollection("projects/gee-base-nina/assets/mapper-soilCondition/vector/response_var_ID")

## Import Terrain and Elevation Data


In [14]:
## Kartverket elevation 10m - Fenoscandinavia
dtm10 = ee.Image("users/rangelandee/NINA/Raster/Fenoscandia_DTM_10m").rename('elevation')

## Høydedata 1m DTM and DSM
dtm_col = ee.ImageCollection("users/vegar/dtm1/dtmcoll")
dsm_col = ee.ImageCollection("users/vegar/dom1/domcoll")

**Mosaic Høydedata's 1m-resolution DTM and DSM ImageCollections**

Mosaic uses the mean pixel value in case pixels overlap. 

In [15]:
# Ensure that the correct projection parameters are used for mosaicing
# original projection of dtm/dsm 
dtm_1m_crs= dtm_col.first().projection()
dtm_1m = ee.Image(dtm_col.mean()).rename('elevation').setDefaultProjection(dtm_1m_crs)
dsm_1m = ee.Image(dsm_col.mean()).rename('elevation').setDefaultProjection(dtm_1m_crs)

## Calculate Slope and Aspect 

In [16]:
# Generate slope and aspect from the dtm10
slope = ee.Terrain.slope(dtm10).rename("slope")
aspect = ee.Terrain.aspect(dtm10).rename("aspect")

## Calculate Canopy Height Module  

In [17]:
# Get canopy height model (CHM) as proxy for vegetation height
chm_1m = dsm_1m.subtract(dtm_1m).rename('CHM'); # CHM = DSM - DTM
chm_1m = chm_1m.setDefaultProjection(dtm_1m_crs)
chm_10m = ee.Image(resample(
    image=chm_1m, 
    method=ee.Reducer.mean(), 
    projection=dtm10.projection(),
    maxPixels=65536))
chm_10m = chm_10m.setDefaultProjection(dtm10.projection())

# check metadata (crs and scale)
# mosaicing sets the scale to the standard "GEE" resolution. 
image_list = [dtm10, dtm_1m, dsm_1m, chm_1m, chm_10m]


## Import Climate Data

In [18]:
# Define original and new names for bioclim variables of interest
bio_namesOriginal = [
    "bio01",
    "bio02",
    "bio03",
    "bio04",
    "bio05",
    "bio06",
    "bio07",
    "bio08",
    "bio09",
    "bio10",
    "bio11",
    "bio12",
    "bio13",
    "bio14",
    "bio15",
    "bio16",
    "bio17",
    "bio18",
    "bio19",
]

bio_namesLong = [
    "temp_mean_annual",
    "temp_diurnal_range",
    "isothermality",
    "temp_seasonality",
    "temp_max_warmestMonth",
    "temp_min_coldestMonth",
    "temp_annual_range",
    "temp_wettestQuart",
    "temp_driestQuart",
    "temp_warmestQuart",
    "temp_coldestQuart",
    "rain_mean_annual",
    "rain_wettestMonth",
    "rain_driestMonth",
    "rain_seasonailty",
    "rain_wettestQuart",
    "rain_driestQuart",
    "rain_warmestQuart",
    "rain_coldestQuart",
]

In [19]:
# Import worldclim dataset with selected variables
# variables are scaled according to guidance for the worldclim dataset on GEE
climateStack = ee.Image("WORLDCLIM/V1/BIO").select(bio_namesOriginal, bio_namesLong)

tempScaleFactor = 0.1
tempSeasonalityScaleFactor = 0.01
precipScaleFactor = 1

climateStackScaled = (
    climateStack.select(bio_namesLong[0:3])
    .multiply(tempScaleFactor)
    .addBands(
        climateStack.select(bio_namesLong[3:4]).multiply(tempSeasonalityScaleFactor)
    )
    .addBands(climateStack.select(bio_namesLong[4:11]).multiply(tempScaleFactor))
    .addBands(climateStack.select(bio_namesLong[11:19]).multiply(precipScaleFactor))
)

## Create Image Stack and Extract Values

In [20]:
combinedStack = dtm10.addBands(slope).addBands(aspect).addBands(chm_10m).addBands(climateStackScaled)

In [21]:
combinedStack.getInfo()

{'type': 'Image',
 'bands': [{'id': 'elevation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [346352, 198156],
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    2.6606302085051987,
    0,
    -8.983152841195215e-05,
    72.05467742298252]},
  {'id': 'slope',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 90},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    2.6606302085051987,
    0,
    -8.983152841195215e-05,
    72.05467742298252]},
  {'id': 'aspect',
   'data_type': {'type': 'PixelType',
    'precision': 'float',
    'min': 0,
    'max': 360},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    2.6606302085051987,
    0,
    -8.983152841195215e-05,
    72.05467742298252]},
  {'id': 'CHM',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [8.983152841195215e-05,
    0,
    

In [22]:
# Add geometry as a property to each feature
def extractVariables(feature, imageDict):
    feature = ee.Feature(feature)
    combinedStack = ee.Image(imageDict.get('combinedStack'))
    dtm10 = ee.Image(imageDict.get('dtm10'))

    # Compute mean values for all bands in the combined image stack
    meanValues = combinedStack.reduceRegion(
        reducer = ee.Reducer.mean(), 
        geometry = feature.geometry(), 
        scale = 10
    )

    # Compute min and max elevation separately
    minMaxElevation = dtm10.reduceRegion(
        reducer = ee.Reducer.minMax(), 
        geometry = feature.geometry(), 
        scale = 10
    )

    # Combine all computed values and set them to the feature
    return feature.set(meanValues).set(minMaxElevation)

In [23]:
# Apply function to each feature in the mires collection
imageDict = ee.Dictionary({
    'combinedStack': combinedStack,
    'dtm10': dtm10
})

miresWithVariables = mires.map(lambda feature: extractVariables(feature, imageDict))

### Import data into Pandas Dataframe

In [24]:
import pandas as pd
df = pd.DataFrame(miresWithVariables.limit(10).getInfo()['features'])

In [25]:
props_df = pd.json_normalize(df['properties'])
new_df = pd.concat([df,props_df], axis=1)
new_df.drop('properties',axis=1,inplace=True)
new_df.drop('geometry',axis=1,inplace=True)
display(new_df)

,type,id,CHM,ID,aspect,avg_ash,avg_cao,avg_dsw,avg_n,avg_pd,...,temp_annual_range,temp_coldestQuart,temp_diurnal_range,temp_driestQuart,temp_max_warmestMonth,temp_mean_annual,temp_min_coldestMonth,temp_seasonality,temp_warmestQuart,temp_wettestQuart
0,Feature,00000000000000000011,0.587308,17,176.014924,2.30,0,0,0,0,...,15.441235,1.758765,4.200000,7.717529,15.200000,6.658765,-0.241235,41.787112,12.058765,4.758765
1,Feature,00000000000000000020,0.453083,32,218.933875,1.30,0,0,0,0,...,16.229819,1.514624,4.515082,7.914624,15.629248,6.614624,-0.600571,43.658635,12.314624,4.584918
2,Feature,00000000000000000037,0.174350,55,151.932638,5.10,0,0,0,0,...,18.200802,0.599798,5.286985,8.361922,16.385388,6.491272,-1.815414,48.373983,12.785388,3.904716
3,Feature,0000000000000000003a,0.178639,58,155.308494,7.40,0,0,0,0,...,18.178995,-0.892710,5.262115,6.649533,14.847144,4.868149,-3.331851,48.175184,11.191594,2.368149
4,Feature,0000000000000000003d,0.488403,61,245.333443,4.85,0,0,0,0,...,18.657610,-0.215143,5.400000,7.602657,15.884857,5.684857,-2.772753,49.130509,12.142448,3.084857
5,Feature,0000000000000000003e,0.250069,62,207.311068,2.60,0,0,0,0,...,19.328810,-0.228810,5.600000,8.000000,16.500000,5.971190,-2.828810,50.613214,12.600000,3.171190
6,Feature,00000000000000000062,0.060884,98,188.774414,10.00,0,0,0,0,...,21.000000,-2.300000,6.300000,6.200000,15.500000,4.100000,-5.500000,52.560000,11.000000,4.700000
7,Feature,00000000000000000063,0.027060,99,150.504218,5.80,0,0,0,0,...,21.177899,-2.703353,6.400000,5.772869,15.174546,3.674546,-6.003353,52.675018,10.672869,4.372869
8,Feature,00000000000000000064,0.056216,100,158.436564,19.20,0,0,0,0,...,21.100000,-2.507652,6.400000,5.892348,15.292348,3.800955,-5.807652,52.612016,10.792348,4.492348
9,Feature,00000000000000000065,0.081411,101,204.728332,24.30,0,0,0,0,...,21.100000,-2.300000,6.400000,6.100000,15.500000,4.000000,-5.600000,52.730000,11.000000,4.700000


## Export the output data 

In [26]:
# Store export options in dictionary
output_folder = 'GEE_output'
export_options = {
  'collection': miresWithVariables,
  'description': 'predictors_terrain_climate',
  'fileFormat': 'CSV',
  'folder': output_folder
  }

In [27]:
import time
# Export table to Google Drive
task = ee.batch.Export.table.toDrive(**export_options)
task.start()

# Get export task status
status = task.status()['state']
while status == 'READY' or status == 'RUNNING':
    time.sleep(5)
    status = task.status()['state']

# Print export location
if status == 'COMPLETED':
    print(f"Export completed. File is stored in ...Google Drive/{output_folder}")
else:
    print("Export failed.")


Export completed. File is stored in ...Google Drive/GEE_output


### Clear Memory 

In [28]:
# Clear data from memory
del miresWithVariables